In [1]:
import os,sys,re,time,math


from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import cluster
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from keras.callbacks import EarlyStopping




import matplotlib as mpl
import numpy as np
import pandas as pd

import sklearn
from matplotlib import pyplot as plt


from keras import backend as K

from keras.optimizers import Adam
from keras.models import *
from keras.layers import *
from keras.utils.np_utils import to_categorical

from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 

#指定第一块GPU可用 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto() 
#不全部占满显存, 按需分配
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

KTF.set_session(sess)

In [3]:
def mean(a):
    return sum(a) / len(a)

In [4]:
def read_svm(file):
    encodings = []
    labels = []
    with open(file) as f:
        records = f.readlines()

    for line in records:
        line = re.sub('\d+:', '', line)
        array = line.strip().split() if line.strip() != '' else None
        encodings.append(array[1:])
        labels.append(int(array[0]))

    return np.array(encodings).astype(float), np.array(labels).astype(int)

In [5]:
def draw_ROC_curve(y_test,y_predict,savepath=None):
    '''
    画ROC曲线
    '''
    false_positive_rate,true_positive_rate,thresholds=roc_curve(y_test, y_predict)
    roc_auc=auc(false_positive_rate, true_positive_rate)
    plt.title('ROC')
    plt.plot(false_positive_rate, true_positive_rate,'b',label='AUC = %0.3f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.ylabel('TPR')
    plt.xlabel('FPR')
    plt.savefig(savepath)
    plt.close(0)

In [6]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [7]:
# 分割并序列编码
AA = 'GAVLIFWYDNEKQMSTCPHR_'
def pep(path, seq_len):
    seqs = open(path).readlines()
    cut = (len(seqs[0].split()[0]) - 1 - seq_len) // 2
    X = [[AA.index(res.upper()) if res.upper() in AA else 0
          for res in (seq.split()[0][cut:-cut] if cut != 0 else seq.split()[0])]
        for seq in seqs if seq.strip() != '']
    y = [int(seq.split()[-1]) for seq in seqs if seq.strip() != '']
    return np.array(X), np.array(y)


In [8]:
def create_cnn_model3(shape, dropout=0.4):
    model = Sequential()

    model.add(Conv1D(128, 8, activation='relu', padding='same',  input_shape=shape))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


In [9]:
def create_cnn_model4(input_length=29,dropout=0.4):
    model = Sequential()
    model.add(Embedding(21, 5, input_length = input_length))
    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


In [10]:
def create_cnn_model5(input_length=29,dropout=0.4):
    model = Sequential()
    model.add(Embedding(21, 5, input_length = input_length))
    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


In [15]:
model = create_cnn_model4(input_length=29)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 29, 5)             105       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 29, 128)           5248      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 14, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 14, 128)           131200    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 128)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 128)            0         
__________

In [85]:
path_train_EGAAC = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_CV/EGAAC/Train_29_EGAAC_gap4.txt'
path_test_EGAAC =  'C:/Users/Crow/Desktop/human_data_12.12/Step_11_IND/EGAAC/Test_29_EGAAC_gap4.txt'

In [10]:
path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Train.txt'
path_test =  'C:/Users/Crow/Desktop/human_data_12.12/Independent.txt'

In [103]:
x_train,y_train = pep(path_train,29)
x_test,y_test = pep(path_test,29)

In [104]:
x_train.shape

(73793, 31)

In [106]:
kf = StratifiedKFold(n_splits = 10)

for train_index, test_index in kf.split(x_train, y_train):

   
    x_train3, x_test3 = x_train[train_index], x_train[test_index]
    y_train3, y_test3 = y_train[train_index], y_train[test_index]
    model = create_cnn_model4(input_length=31)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    callbacks_list = [early_stopping]
    model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 256,shuffle=True,
         callbacks=callbacks_list, verbose=1)

pre = model.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test,pre,pos_label=1)
pre1 = model.predict_classes(x_test)
precision,recall,SN,SP,GM,TP,TN,FP,FN = performance(y_test,pre1)
print(model.evaluate(x_test, y_test, batch_size=256))
print("ACC:  %f "  %accuracy_score(y_test,pre1))
print("Sn: %f" %SN)
print("Sp: %f" %SP)
print("MCC: %f " %matthews_corrcoef(y_test,pre1))
print("AUC: %f" % auc(fpr, tpr))

Train on 66413 samples, validate on 7380 samples
Epoch 1/20
66413/66413 [==============================] - 101s 2ms/step - loss: 0.3100 - acc: 0.9078 - val_loss: 0.2967 - val_acc: 0.9093
Epoch 2/20
66413/66413 [==============================] - 12s 177us/step - loss: 0.2861 - acc: 0.9094 - val_loss: 0.2749 - val_acc: 0.9093
Epoch 3/20
66413/66413 [==============================] - 12s 180us/step - loss: 0.2624 - acc: 0.9094 - val_loss: 0.2472 - val_acc: 0.9093
Epoch 4/20
66413/66413 [==============================] - 12s 174us/step - loss: 0.2504 - acc: 0.9093 - val_loss: 0.2381 - val_acc: 0.9095
Epoch 5/20
66413/66413 [==============================] - 12s 178us/step - loss: 0.2427 - acc: 0.9088 - val_loss: 0.2286 - val_acc: 0.9099
Epoch 6/20
66413/66413 [==============================] - 12s 175us/step - loss: 0.2363 - acc: 0.9091 - val_loss: 0.2241 - val_acc: 0.9093
Epoch 7/20
66413/66413 [==============================] - 12s 174us/step - loss: 0.2330 - acc: 0.9090 - val_loss: 0.22

66415/66415 [==============================] - 12s 181us/step - loss: 0.2486 - acc: 0.9091 - val_loss: 0.2453 - val_acc: 0.9095
Epoch 5/20
66415/66415 [==============================] - 12s 183us/step - loss: 0.2405 - acc: 0.9092 - val_loss: 0.2504 - val_acc: 0.9091
Epoch 6/20
66415/66415 [==============================] - 12s 180us/step - loss: 0.2369 - acc: 0.9091 - val_loss: 0.2408 - val_acc: 0.9080
Epoch 7/20
66415/66415 [==============================] - 12s 178us/step - loss: 0.2329 - acc: 0.9097 - val_loss: 0.2345 - val_acc: 0.9088
Epoch 8/20
66415/66415 [==============================] - 12s 180us/step - loss: 0.2289 - acc: 0.9094 - val_loss: 0.2333 - val_acc: 0.9084
Epoch 9/20
66415/66415 [==============================] - 12s 180us/step - loss: 0.2256 - acc: 0.9105 - val_loss: 0.2323 - val_acc: 0.9092
Epoch 10/20
66415/66415 [==============================] - 12s 180us/step - loss: 0.2237 - acc: 0.9106 - val_loss: 0.2310 - val_acc: 0.9082
Epoch 11/20
66415/66415 [============

In [15]:
# embedding: 5 
model.predict(x_test)
model.predict_function()



17980/17980 [==============================] - 0s 9us/step
[0.21914273882336557, 0.9150166852588251]
ACC:  0.915017 
Sn: 0.163857
Sp: 0.982542
MCC: 0.237892 
AUC: 0.856130


In [11]:
def create_cnn_model3(shape, dropout=0.4):
    model = Sequential()

    model.add(Conv1D(128, 8, activation='relu', padding='same',  input_shape=shape))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Conv1D(128, 8, activation='relu', padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


In [10]:
name = 'CKSAAP'
gap = '_gap4'
# 读取数据


path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_CV/'+ name +'/Train_29_'+ name + gap +'.txt'
path_test = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_IND/'+ name +'/Test_29_'+ name + gap +'.txt'

train = read_svm(path_train)
test = read_svm(path_test)


x_train = train[0]
y_train = train[1]

x_test = test[0]
y_test = test[1]

x_train = np.expand_dims(x_train, axis=2) 
x_test = np.expand_dims(x_test, axis=2) 

shape = x_train.shape[1:]

# path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Train.txt'
# path_test =  'C:/Users/Crow/Desktop/human_data_12.12/Independent.txt'

# x_train,y_train = pep(path_train,29-2)
# x_test,y_test = pep(path_test,29-2)



shape = x_train.shape[1:]

kf = KFold(n_splits = 10,random_state=5,shuffle=True)
j = 1 
for train_index, test_index in kf.split(x_train):
    x_train3, x_test3 = x_train[train_index], x_train[test_index]
    y_train3, y_test3 = y_train[train_index], y_train[test_index]
    
    model = create_cnn_model3(shape=shape)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    callbacks_list = [early_stopping]
    model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 256,shuffle=True,
         callbacks=callbacks_list, verbose=1)
    
    
    
    
    test_pred_proba = model.predict(x_test3)
    fpr, tpr, thresholds = roc_curve(y_test3,test_pred_proba,pos_label=1)
    #print("ACC:  %f "  %accuracy_score(y_test3,test_pred))
    print("AUC: %f" % auc(fpr, tpr))
    #print("MCC: %f " %matthews_corrcoef(y_test3,test_pred))
    fw = open('C:/Users/Crow/Desktop/result/29_kfold_CNN_'+ name + gap+'_result_'+ str(j) +'.txt','w')
   
    for t in range(0,len(test_pred_proba)):
        fw.write(str(test_pred_proba[t][0]))
        fw.write('\t')
        fw.write(str(y_test3[t]))
        fw.write('\n')
    fw.close()

    
    if j == 10:        
        model.save('C:/Users/Crow/Desktop/result/model/CNN_kfold_'+ name + gap +'.h5')
        
        test_pred_proba = model.predict(x_test)
        fpr, tpr, thresholds = roc_curve(y_test,test_pred_proba,pos_label=1)
        print("总AUC: %f" % auc(fpr, tpr))
        fw = open('C:/Users/Crow/Desktop/result/29_kfold_CNN_'+ name + gap +'_result.txt','w')
        for t in range(0,len(test_pred_proba)):
            fw.write(str(test_pred_proba[t][0]))
            fw.write('\t')
            fw.write(str(y_test[t]))
            fw.write('\n')
        fw.close()
    j+=1
    

Train on 66413 samples, validate on 7380 samples
Epoch 1/20
66413/66413 [==============================] - 27s 409us/step - loss: 0.3129 - acc: 0.9091 - val_loss: 0.3079 - val_acc: 0.9077
Epoch 2/20
66413/66413 [==============================] - 25s 379us/step - loss: 0.3021 - acc: 0.9096 - val_loss: 0.3043 - val_acc: 0.9077
Epoch 3/20
66413/66413 [==============================] - 25s 377us/step - loss: 0.2952 - acc: 0.9096 - val_loss: 0.3036 - val_acc: 0.9077
Epoch 4/20
66413/66413 [==============================] - 25s 378us/step - loss: 0.2920 - acc: 0.9096 - val_loss: 0.2948 - val_acc: 0.90772s - los
Epoch 5/20
66413/66413 [==============================] - 25s 373us/step - loss: 0.2913 - acc: 0.9096 - val_loss: 0.2945 - val_acc: 0.9077
Epoch 6/20
66413/66413 [==============================] - 25s 371us/step - loss: 0.2887 - acc: 0.9095 - val_loss: 0.2916 - val_acc: 0.9077
Epoch 7/20
66413/66413 [==============================] - 25s 371us/step - loss: 0.2873 - acc: 0.9095 - val_l

66414/66414 [==============================] - 25s 377us/step - loss: 0.2965 - acc: 0.9090 - val_loss: 0.2889 - val_acc: 0.9125
Epoch 4/20
66414/66414 [==============================] - 25s 379us/step - loss: 0.2938 - acc: 0.9090 - val_loss: 0.2912 - val_acc: 0.9125 1s - loss:
Epoch 5/20
66414/66414 [==============================] - 25s 379us/step - loss: 0.2918 - acc: 0.9090 - val_loss: 0.2877 - val_acc: 0.9125s - loss: 0.2889 - acc: 0.910 - ETA: 6s - loss: 0.2892 - acc: 0. - ETA: 5s - - E
Epoch 6/20
66414/66414 [==============================] - 25s 379us/step - loss: 0.2890 - acc: 0.9090 - val_loss: 0.2839 - val_acc: 0.9125 loss: 0. - ETA: 17s  - ETA: 12s - loss: 0.2916 - acc: 0.90 - ETA: 12s - loss: 0.2914 - acc - ETA: 11s  - ETA: 10s - loss: 0.2920 - acc: 0.90 - ETA: 10s - loss: - ETA: 8s - loss: 0.2908 - acc - ETA: 7s - loss: 0.2909 - acc: 0.908 - ETA: 7s  - ETA: 5s - loss - ETA: 3s - loss: 0.2894 - acc: 0. - ETA: 2s - loss: 0.2893 - acc: 0.908 - ETA: 2
Epoch 7/20
66414/66414 [=

In [15]:
test_pred_proba[t][0]

0.10685588

In [12]:
# 重新训练CNN word Embedding

name = 'EGAAC'
gap = '_gap4'
auc_mean=[]
# path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Train.txt'
# path_test =  'C:/Users/Crow/Desktop/human_data_12.12/Independent.txt'
# x_train,y_train = pep(path_train,29-2)
# x_test,y_test = pep(path_test,29-2)

path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_CV/'+ name +'/Train_29_'+ name + gap +'.txt'
path_test = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_IND/'+ name +'/Test_29_'+ name + gap +'.txt'

train = read_svm(path_train)
test = read_svm(path_test)


x_train = train[0]
y_train = train[1]

x_test = test[0]
y_test = test[1]

x_train = np.expand_dims(x_train, axis=2) 
x_test = np.expand_dims(x_test, axis=2) 

shape = x_train.shape[1:]

kf = KFold(n_splits = 10,random_state=5,shuffle=True)
j = 1 
for train_index, test_index in kf.split(x_train):
    x_train3, x_test3 = x_train[train_index], x_train[test_index]
    y_train3, y_test3 = y_train[train_index], y_train[test_index]
    
#     model = create_cnn_model5(input_length=29)
    
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5)
#     callbacks_list = [early_stopping]
#     model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 256,shuffle=True,
#          callbacks=callbacks_list, verbose=1)
    model = create_cnn_model3(shape=shape,dropout=0.6)
    
    #filepath='C:/Users/Crow/Desktop/result/re_CNN3/model/'+ str(j) +'checkpoint-{val_loss:.2f}-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'
    filepath='C:/Users/Crow/Desktop/new_result/CNN/model/29_kfold_CNN_'+ name + gap+'_'+ str(j) +'.hdf5'
    #filepath="C:/Users/Crow/Desktop/result/re_CNN/model/weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,mode='auto', period=50)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=50)
    callbacks_list = [early_stopping,checkpoint]
    model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 1000, batch_size = 256,
              shuffle=True,callbacks=callbacks_list, verbose=1)
    
    
    
    
    test_pred_proba = model.predict(x_test3)
    fpr, tpr, thresholds = roc_curve(y_test3,test_pred_proba,pos_label=1)
    #print("ACC:  %f "  %accuracy_score(y_test3,test_pred))
    print("AUC: %f" % auc(fpr, tpr))
    auc_mean.append(auc(fpr, tpr))
    #print("MCC: %f " %matthews_corrcoef(y_test3,test_pred))
    fw = open('C:/Users/Crow/Desktop/new_result/CNN/29_kfold_CNN_'+ name + gap+'_result_'+ str(j) +'.txt','w')
    for t in range(0,len(test_pred_proba)):
        fw.write(str(test_pred_proba[t][0]))
        fw.write('\t')
        fw.write(str(y_test3[t]))
        fw.write('\n')
    fw.close()
    
    fw = open('C:/Users/Crow/Desktop/new_result/CNN/29_kfold_CNN_'+ name + gap+'_test_'+ str(j) +'.txt','w')
    for t in range(0,len(y_test3)):
        fw.write(str(y_test3[t]))
        fw.write('\n')
    fw.close()
    
    if j == 10:
        print(auc_mean)
        print(print("CV AUC: %f" % mean(auc_mean)))
#         model.save('C:/Users/Crow/Desktop/result/re_CNN/model/CNN_kfold_'+ name + gap +'.h5') 
        
#         test_pred_proba = model.predict(x_test)
#         fpr, tpr, thresholds = roc_curve(y_test,test_pred_proba,pos_label=1)
#         print("总AUC: %f" % auc(fpr, tpr))
#         fw = open('C:/Users/Crow/Desktop/result/re_CNN/29_kfold_CNN_'+ name + gap +'_result.txt','w')
#         for t in range(0,len(test_pred_proba)):
#             fw.write(str(test_pred_proba[t][0]))
#             fw.write('\t')
#             fw.write(str(y_test[t]))
#             fw.write('\n') 
#         fw.close()
    j+=1

Train on 66413 samples, validate on 7380 samples
Epoch 1/1000
66413/66413 [==============================] - 11s 166us/step - loss: 0.3062 - acc: 0.9076 - val_loss: 0.3094 - val_acc: 0.9077
Epoch 2/1000
66413/66413 [==============================] - 8s 117us/step - loss: 0.2966 - acc: 0.9096 - val_loss: 0.2983 - val_acc: 0.9077
Epoch 3/1000
66413/66413 [==============================] - 8s 115us/step - loss: 0.2909 - acc: 0.9096 - val_loss: 0.3258 - val_acc: 0.9077.2919 - acc: 0
Epoch 4/1000
66413/66413 [==============================] - 8s 116us/step - loss: 0.2886 - acc: 0.9096 - val_loss: 0.2899 - val_acc: 0.9077
Epoch 5/1000
66413/66413 [==============================] - 8s 115us/step - loss: 0.2840 - acc: 0.9096 - val_loss: 0.2933 - val_acc: 0.9077
Epoch 6/1000
66413/66413 [==============================] - 8s 116us/step - loss: 0.2824 - acc: 0.9096 - val_loss: 0.2829 - val_acc: 0.9077
Epoch 7/1000
66413/66413 [==============================] - 8s 116us/step - loss: 0.2815 - acc: 

66413/66413 [==============================] - 8s 118us/step - loss: 0.2583 - acc: 0.9096 - val_loss: 0.2692 - val_acc: 0.9081
Epoch 114/1000
66413/66413 [==============================] - 8s 119us/step - loss: 0.2579 - acc: 0.9098 - val_loss: 0.2721 - val_acc: 0.9080
Epoch 115/1000
66413/66413 [==============================] - 8s 118us/step - loss: 0.2565 - acc: 0.9096 - val_loss: 0.2752 - val_acc: 0.9087
Epoch 116/1000
66413/66413 [==============================] - 8s 116us/step - loss: 0.2577 - acc: 0.9100 - val_loss: 0.2687 - val_acc: 0.9080
Epoch 117/1000
66413/66413 [==============================] - 8s 116us/step - loss: 0.2575 - acc: 0.9103 - val_loss: 0.2688 - val_acc: 0.9079
Epoch 118/1000
66413/66413 [==============================] - 8s 117us/step - loss: 0.2568 - acc: 0.9098 - val_loss: 0.2711 - val_acc: 0.9077
Epoch 119/1000
66413/66413 [==============================] - 8s 117us/step - loss: 0.2587 - acc: 0.9097 - val_loss: 0.2710 - val_acc: 0.9075
Epoch 120/1000
66413/

66413/66413 [==============================] - 8s 118us/step - loss: 0.2681 - acc: 0.9090 - val_loss: 0.2645 - val_acc: 0.9127
Epoch 43/1000
66413/66413 [==============================] - 8s 118us/step - loss: 0.2682 - acc: 0.9094 - val_loss: 0.2723 - val_acc: 0.9129
Epoch 44/1000
66413/66413 [==============================] - 8s 119us/step - loss: 0.2675 - acc: 0.9088 - val_loss: 0.2690 - val_acc: 0.9127
Epoch 45/1000
66413/66413 [==============================] - 8s 116us/step - loss: 0.2678 - acc: 0.9092 - val_loss: 0.2669 - val_acc: 0.9127
Epoch 46/1000
66413/66413 [==============================] - 8s 120us/step - loss: 0.2673 - acc: 0.9088 - val_loss: 0.2717 - val_acc: 0.9130
Epoch 47/1000
66413/66413 [==============================] - 8s 119us/step - loss: 0.2672 - acc: 0.9090 - val_loss: 0.2720 - val_acc: 0.9126
Epoch 48/1000
66413/66413 [==============================] - 8s 118us/step - loss: 0.2680 - acc: 0.9091 - val_loss: 0.2701 - val_acc: 0.9130
Epoch 49/1000
66413/66413 [

Epoch 99/1000
66413/66413 [==============================] - 8s 119us/step - loss: 0.2604 - acc: 0.9092 - val_loss: 0.2616 - val_acc: 0.9131
Epoch 100/1000
66413/66413 [==============================] - 8s 118us/step - loss: 0.2610 - acc: 0.9090 - val_loss: 0.2627 - val_acc: 0.9131

Epoch 00100: saving model to C:/Users/Crow/Desktop/new_result/CNN/model/29_kfold_CNN_EGAAC_gap4_2.hdf5
Epoch 101/1000
66413/66413 [==============================] - 8s 118us/step - loss: 0.2609 - acc: 0.9093 - val_loss: 0.2679 - val_acc: 0.9131
Epoch 102/1000
66413/66413 [==============================] - 8s 117us/step - loss: 0.2594 - acc: 0.9091 - val_loss: 0.2609 - val_acc: 0.9129
Epoch 103/1000
66413/66413 [==============================] - 8s 119us/step - loss: 0.2601 - acc: 0.9091 - val_loss: 0.2618 - val_acc: 0.9131
Epoch 104/1000
66413/66413 [==============================] - 8s 115us/step - loss: 0.2613 - acc: 0.9092 - val_loss: 0.2617 - val_acc: 0.9126
Epoch 105/1000
66413/66413 [=================

Epoch 155/1000
66413/66413 [==============================] - 31s 464us/step - loss: 0.2554 - acc: 0.9091 - val_loss: 0.2625 - val_acc: 0.9130
Epoch 156/1000
66413/66413 [==============================] - 31s 468us/step - loss: 0.2562 - acc: 0.9089 - val_loss: 0.2618 - val_acc: 0.9129
Epoch 157/1000
66413/66413 [==============================] - 31s 469us/step - loss: 0.2557 - acc: 0.9093 - val_loss: 0.2619 - val_acc: 0.9123
Epoch 158/1000
66413/66413 [==============================] - 30s 455us/step - loss: 0.2560 - acc: 0.9097 - val_loss: 0.2619 - val_acc: 0.9129
Epoch 159/1000
66413/66413 [==============================] - 31s 468us/step - loss: 0.2564 - acc: 0.9093 - val_loss: 0.2588 - val_acc: 0.9123
Epoch 160/1000
66413/66413 [==============================] - 31s 462us/step - loss: 0.2575 - acc: 0.9092 - val_loss: 0.2648 - val_acc: 0.9129
Epoch 161/1000
66413/66413 [==============================] - 31s 463us/step - loss: 0.2560 - acc: 0.9096 - val_loss: 0.2590 - val_acc: 0.9129

...

Epoch 51/1000
66414/66414 [==============================] - 23s 341us/step - loss: 0.2646 - acc: 0.9091 - val_loss: 0.2771 - val_acc: 0.9093
Epoch 52/1000
66414/66414 [==============================] - 23s 343us/step - loss: 0.2635 - acc: 0.9094 - val_loss: 0.2787 - val_acc: 0.9089
Epoch 53/1000
66414/66414 [==============================] - 23s 346us/step - loss: 0.2635 - acc: 0.9096 - val_loss: 0.2757 - val_acc: 0.90880s - loss: 0.2635 - acc:  - ETA: 0s - loss: 0.2635 - acc: 0.9
Epoch 54/1000
66414/66414 [==============================] - 23s 343us/step - loss: 0.2647 - acc: 0.9092 - val_loss: 0.2710 - val_acc: 0.9088 ETA: 4s - loss: 0.2637  - ETA: 0s - loss: 0.2654 - acc: 0.9
Epoch 55/1000
66414/66414 [==============================] - 23s 348us/step - loss: 0.2647 - acc: 0.9095 - val_loss: 0.2756 - val_acc: 0.9087
Epoch 56/1000
66414/66414 [==============================] - 23s 348us/step - loss: 0.2649 - acc: 0.9094 - val_loss: 0.2734 - val_acc: 0.9087
Epoch 57/1000
66414/66414 [

66414/66414 [==============================] - 23s 346us/step - loss: 0.2555 - acc: 0.9100 - val_loss: 0.2694 - val_acc: 0.9088
Epoch 149/1000
66414/66414 [==============================] - 23s 348us/step - loss: 0.2549 - acc: 0.9098 - val_loss: 0.2730 - val_acc: 0.9092
Epoch 150/1000
66414/66414 [==============================] - 23s 350us/step - loss: 0.2553 - acc: 0.9095 - val_loss: 0.2723 - val_acc: 0.9092

Epoch 00150: saving model to C:/Users/Crow/Desktop/new_result/CNN/model/29_kfold_CNN_EGAAC_gap4_8.hdf5
Epoch 151/1000
66414/66414 [==============================] - 23s 346us/step - loss: 0.2556 - acc: 0.9091 - val_loss: 0.2715 - val_acc: 0.9087
Epoch 152/1000
66414/66414 [==============================] - 23s 346us/step - loss: 0.2557 - acc: 0.9095 - val_loss: 0.2717 - val_acc: 0.9089.25 - ETA: 9s 
Epoch 153/1000
66414/66414 [==============================] - 23s 347us/step - loss: 0.2549 - acc: 0.9098 - val_loss: 0.2696 - val_acc: 0.9091
Epoch 154/1000
66414/66414 [===========

66414/66414 [==============================] - 23s 350us/step - loss: 0.2641 - acc: 0.9092 - val_loss: 0.2677 - val_acc: 0.9088
Epoch 68/1000
66414/66414 [==============================] - 24s 357us/step - loss: 0.2635 - acc: 0.9096 - val_loss: 0.2607 - val_acc: 0.9091
Epoch 69/1000
66414/66414 [==============================] - 23s 351us/step - loss: 0.2631 - acc: 0.9094 - val_loss: 0.2667 - val_acc: 0.9088
Epoch 70/1000
66414/66414 [==============================] - 23s 350us/step - loss: 0.2638 - acc: 0.9092 - val_loss: 0.2612 - val_acc: 0.9091 0.2656  - ETA: 3s - loss: 0.2654 - acc: - ETA: 2s 
Epoch 71/1000
66414/66414 [==============================] - 23s 351us/step - loss: 0.2644 - acc: 0.9094 - val_loss: 0.2653 - val_acc: 0.90910.2644 - acc: 0.90
Epoch 72/1000
66414/66414 [==============================] - 23s 347us/step - loss: 0.2632 - acc: 0.9094 - val_loss: 0.2675 - val_acc: 0.9088
Epoch 73/1000
66414/66414 [==============================] - 23s 347us/step - loss: 0.2630 - 

66414/66414 [==============================] - 17s 261us/step - loss: 0.2673 - acc: 0.9083 - val_loss: 0.2561 - val_acc: 0.9162
Epoch 53/1000
66414/66414 [==============================] - 17s 261us/step - loss: 0.2667 - acc: 0.9083 - val_loss: 0.2652 - val_acc: 0.9161
Epoch 54/1000
66414/66414 [==============================] - 17s 261us/step - loss: 0.2662 - acc: 0.9087 - val_loss: 0.2695 - val_acc: 0.9157
Epoch 55/1000
66414/66414 [==============================] - 17s 261us/step - loss: 0.2665 - acc: 0.9083 - val_loss: 0.2649 - val_acc: 0.9157
Epoch 56/1000
66414/66414 [==============================] - 17s 262us/step - loss: 0.2663 - acc: 0.9087 - val_loss: 0.2667 - val_acc: 0.9158
Epoch 57/1000
66414/66414 [==============================] - 17s 261us/step - loss: 0.2652 - acc: 0.9086 - val_loss: 0.2632 - val_acc: 0.9161
Epoch 58/1000
66414/66414 [==============================] - 17s 261us/step - loss: 0.2646 - acc: 0.9085 - val_loss: 0.2590 - val_acc: 0.9160
Epoch 59/1000
66414/

Epoch 109/1000
66414/66414 [==============================] - 17s 262us/step - loss: 0.2591 - acc: 0.9092 - val_loss: 0.2619 - val_acc: 0.9158
Epoch 110/1000
66414/66414 [==============================] - 18s 264us/step - loss: 0.2605 - acc: 0.9088 - val_loss: 0.2549 - val_acc: 0.9158
Epoch 111/1000
66414/66414 [==============================] - 17s 263us/step - loss: 0.2590 - acc: 0.9093 - val_loss: 0.2593 - val_acc: 0.9158
Epoch 112/1000
66414/66414 [==============================] - 17s 262us/step - loss: 0.2601 - acc: 0.9086 - val_loss: 0.2558 - val_acc: 0.9156
Epoch 113/1000
66414/66414 [==============================] - 17s 262us/step - loss: 0.2594 - acc: 0.9086 - val_loss: 0.2553 - val_acc: 0.9158
Epoch 114/1000
66414/66414 [==============================] - 18s 267us/step - loss: 0.2583 - acc: 0.9086 - val_loss: 0.2579 - val_acc: 0.9160
Epoch 115/1000
66414/66414 [==============================] - 18s 265us/step - loss: 0.2583 - acc: 0.9089 - val_loss: 0.2610 - val_acc: 0.9158

In [11]:
# 5种特征 CNN
# 重新训练CNN word Embedding

name = 'CKSAAP'
gap = '_gap4'
auc_mean=[]
# path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Train.txt'
# path_test =  'C:/Users/Crow/Desktop/human_data_12.12/Independent.txt'
# x_train,y_train = pep(path_train,29-2)
# x_test,y_test = pep(path_test,29-2)

path_train = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_CV/'+ name +'/Train_29_'+ name + gap +'.txt'
path_test = 'C:/Users/Crow/Desktop/human_data_12.12/Step_11_IND/'+ name +'/Test_29_'+ name + gap +'.txt'

train = read_svm(path_train)
test = read_svm(path_test)


x_train = train[0]
y_train = train[1]

x_test = test[0]
y_test = test[1]

x_train = np.expand_dims(x_train, axis=2) 
x_test = np.expand_dims(x_test, axis=2) 

shape = x_train.shape[1:]

kf = KFold(n_splits = 10,random_state=5,shuffle=True)
j = 1 
for train_index, test_index in kf.split(x_train):
    x_train3, x_test3 = x_train[train_index], x_train[test_index]
    y_train3, y_test3 = y_train[train_index], y_train[test_index]
    
#     model = create_cnn_model5(input_length=29)
    
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5)
#     callbacks_list = [early_stopping]
#     model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 20, batch_size = 256,shuffle=True,
#          callbacks=callbacks_list, verbose=1)
    model = create_cnn_model3(shape=shape,dropout=0.6)
    
    #filepath='C:/Users/Crow/Desktop/result/re_CNN3/model/'+ str(j) +'checkpoint-{val_loss:.2f}-{epoch:02d}e-val_acc_{val_acc:.2f}.hdf5'
    filepath='C:/Users/Crow/Desktop/new_result/CNN/model/29_kfold_CNN_'+ name + gap+'_'+ str(j) +'.hdf5'
    #filepath="C:/Users/Crow/Desktop/result/re_CNN/model/weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False,mode='auto', period=50)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=50)
    callbacks_list = [early_stopping,checkpoint]
    model.fit(x_train3, y_train3, validation_data = (x_test3, y_test3), epochs = 1000, batch_size = 256,
              shuffle=True,callbacks=callbacks_list, verbose=1)
    
    
    
    
    test_pred_proba = model.predict(x_test3)
    fpr, tpr, thresholds = roc_curve(y_test3,test_pred_proba,pos_label=1)
    #print("ACC:  %f "  %accuracy_score(y_test3,test_pred))
    print("AUC: %f" % auc(fpr, tpr))
    auc_mean.append(auc(fpr, tpr))
    #print("MCC: %f " %matthews_corrcoef(y_test3,test_pred))
    fw = open('C:/Users/Crow/Desktop/new_result/CNN/29_kfold_CNN_'+ name + gap+'_result_'+ str(j) +'.txt','w')
    for t in range(0,len(test_pred_proba)):
        fw.write(str(test_pred_proba[t][0]))
        fw.write('\t')
        fw.write(str(y_test3[t]))
        fw.write('\n')
    fw.close()
    
    fw = open('C:/Users/Crow/Desktop/new_result/CNN/29_kfold_CNN_'+ name + gap+'_test_'+ str(j) +'.txt','w')
    for t in range(0,len(y_test3)):
        fw.write(str(y_test3[t]))
        fw.write('\n')
    fw.close()
    
    if j == 10:
        print(auc_mean)
        print(print("CV AUC: %f" % mean(auc_mean)))
#         model.save('C:/Users/Crow/Desktop/result/re_CNN/model/CNN_kfold_'+ name + gap +'.h5') 
        
#         test_pred_proba = model.predict(x_test)
#         fpr, tpr, thresholds = roc_curve(y_test,test_pred_proba,pos_label=1)
#         print("总AUC: %f" % auc(fpr, tpr))
#         fw = open('C:/Users/Crow/Desktop/result/re_CNN/29_kfold_CNN_'+ name + gap +'_result.txt','w')
#         for t in range(0,len(test_pred_proba)):
#             fw.write(str(test_pred_proba[t][0]))
#             fw.write('\t')
#             fw.write(str(y_test[t]))
#             fw.write('\n') 
#         fw.close()
    j+=1

Train on 66413 samples, validate on 7380 samples
Epoch 1/1000
66413/66413 [==============================] - 27s 408us/step - loss: 0.3137 - acc: 0.9069 - val_loss: 0.3153 - val_acc: 0.9077
Epoch 2/1000
66413/66413 [==============================] - 24s 366us/step - loss: 0.3026 - acc: 0.9096 - val_loss: 0.3038 - val_acc: 0.9077
Epoch 3/1000
66413/66413 [==============================] - 24s 367us/step - loss: 0.2999 - acc: 0.9096 - val_loss: 0.3007 - val_acc: 0.9077
Epoch 4/1000
66413/66413 [==============================] - 24s 367us/step - loss: 0.2948 - acc: 0.9096 - val_loss: 0.3056 - val_acc: 0.9077
Epoch 5/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2936 - acc: 0.9096 - val_loss: 0.2932 - val_acc: 0.9077loss: 0.2935 - ac
Epoch 6/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2919 - acc: 0.9096 - val_loss: 0.3021 - val_acc: 0.9077 - loss: 0.2920 -  - ETA: 6
Epoch 7/1000
66413/66413 [==============================] - 2

66413/66413 [==============================] - 24s 368us/step - loss: 0.2505 - acc: 0.9105 - val_loss: 0.2864 - val_acc: 0.9077
Epoch 113/1000
66413/66413 [==============================] - 25s 369us/step - loss: 0.2490 - acc: 0.9112 - val_loss: 0.2895 - val_acc: 0.9068
Epoch 114/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2488 - acc: 0.9116 - val_loss: 0.2860 - val_acc: 0.9073s: 0.24 - ETA: 1s - loss: 0
Epoch 115/1000
66413/66413 [==============================] - 25s 370us/step - loss: 0.2504 - acc: 0.9116 - val_loss: 0.2844 - val_acc: 0.9081
Epoch 116/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2514 - acc: 0.9113 - val_loss: 0.2882 - val_acc: 0.9075
Epoch 117/1000
66413/66413 [==============================] - 25s 369us/step - loss: 0.2483 - acc: 0.9117 - val_loss: 0.2853 - val_acc: 0.9072
Epoch 118/1000
66413/66413 [==============================] - 25s 369us/step - loss: 0.2475 - acc: 0.9117 - val_loss: 0.2844 - val

66413/66413 [==============================] - 24s 369us/step - loss: 0.2587 - acc: 0.9099 - val_loss: 0.2881 - val_acc: 0.9099
Epoch 73/1000
66413/66413 [==============================] - 24s 369us/step - loss: 0.2564 - acc: 0.9103 - val_loss: 0.2843 - val_acc: 0.9106
Epoch 74/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2570 - acc: 0.9099 - val_loss: 0.2781 - val_acc: 0.9110
Epoch 75/1000
66413/66413 [==============================] - 24s 369us/step - loss: 0.2556 - acc: 0.9097 - val_loss: 0.2850 - val_acc: 0.9103
Epoch 76/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2553 - acc: 0.9101 - val_loss: 0.2854 - val_acc: 0.9104
Epoch 77/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2541 - acc: 0.9105 - val_loss: 0.2830 - val_acc: 0.9106
Epoch 78/1000
66413/66413 [==============================] - 24s 368us/step - loss: 0.2542 - acc: 0.9104 - val_loss: 0.2857 - val_acc: 0.9104
Epoch 79/1000
66413/

66413/66413 [==============================] - 24s 367us/step - loss: 0.2631 - acc: 0.9101 - val_loss: 0.2820 - val_acc: 0.9088: - ETA: 2
Epoch 68/1000
66413/66413 [==============================] - 24s 366us/step - loss: 0.2635 - acc: 0.9095 - val_loss: 0.2927 - val_acc: 0.9088
Epoch 69/1000
66413/66413 [==============================] - 24s 366us/step - loss: 0.2629 - acc: 0.9101 - val_loss: 0.2836 - val_acc: 0.9088
Epoch 70/1000
66413/66413 [==============================] - 24s 366us/step - loss: 0.2630 - acc: 0.9096 - val_loss: 0.2844 - val_acc: 0.9087
Epoch 71/1000
66413/66413 [==============================] - 24s 367us/step - loss: 0.2615 - acc: 0.9099 - val_loss: 0.2880 - val_acc: 0.9085 0s - loss: 0.2615 - acc: 0.909
Epoch 72/1000
66413/66413 [==============================] - 24s 367us/step - loss: 0.2606 - acc: 0.9099 - val_loss: 0.2827 - val_acc: 0.9084
Epoch 73/1000
66413/66413 [==============================] - 24s 366us/step - loss: 0.2610 - acc: 0.9102 - val_loss: 0.27

66414/66414 [==============================] - 24s 367us/step - loss: 0.2564 - acc: 0.9102 - val_loss: 0.2868 - val_acc: 0.9050
Epoch 67/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2565 - acc: 0.9099 - val_loss: 0.2958 - val_acc: 0.9028
Epoch 68/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2561 - acc: 0.9107 - val_loss: 0.2879 - val_acc: 0.9038
Epoch 69/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2544 - acc: 0.9108 - val_loss: 0.2857 - val_acc: 0.9047
Epoch 70/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2537 - acc: 0.9114 - val_loss: 0.2899 - val_acc: 0.9038
Epoch 71/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2544 - acc: 0.9115 - val_loss: 0.2948 - val_acc: 0.9034
Epoch 72/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2551 - acc: 0.9110 - val_loss: 0.2919 - val_acc: 0.9030
Epoch 73/1000
66414/

...

66414/66414 [==============================] - 24s 367us/step - loss: 0.2509 - acc: 0.9108 - val_loss: 0.2798 - val_acc: 0.9091
Epoch 91/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2525 - acc: 0.9100 - val_loss: 0.2783 - val_acc: 0.9100
Epoch 92/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2501 - acc: 0.9120 - val_loss: 0.2772 - val_acc: 0.9104
Epoch 93/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2516 - acc: 0.9106 - val_loss: 0.2803 - val_acc: 0.9106
Epoch 94/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2496 - acc: 0.9110 - val_loss: 0.2790 - val_acc: 0.9097
Epoch 95/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2500 - acc: 0.9118 - val_loss: 0.2763 - val_acc: 0.9102
Epoch 96/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2482 - acc: 0.9121 - val_loss: 0.2745 - val_acc: 0.9091
Epoch 97/1000
66414/

Epoch 146/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2368 - acc: 0.9135 - val_loss: 0.2781 - val_acc: 0.9084
Epoch 147/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2383 - acc: 0.9130 - val_loss: 0.2789 - val_acc: 0.9087
Epoch 148/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2358 - acc: 0.9144 - val_loss: 0.2805 - val_acc: 0.9077
Epoch 149/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2378 - acc: 0.9135 - val_loss: 0.2804 - val_acc: 0.9073s - loss: 0.2370 - ac
Epoch 150/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2358 - acc: 0.9139 - val_loss: 0.2811 - val_acc: 0.9068

Epoch 00150: saving model to C:/Users/Crow/Desktop/new_result/CNN/model/29_kfold_CNN_CKSAAP_gap4_8.hdf5
Epoch 151/1000
66414/66414 [==============================] - 24s 365us/step - loss: 0.2360 - acc: 0.9130 - val_loss: 0.2771 - val_acc: 0.9088
Epoch 152/1000
6

66414/66414 [==============================] - 25s 370us/step - loss: 0.2472 - acc: 0.9117 - val_loss: 0.2798 - val_acc: 0.9081
Epoch 102/1000
66414/66414 [==============================] - 25s 370us/step - loss: 0.2470 - acc: 0.9118 - val_loss: 0.2789 - val_acc: 0.9073
Epoch 103/1000
66414/66414 [==============================] - 25s 370us/step - loss: 0.2459 - acc: 0.9114 - val_loss: 0.2778 - val_acc: 0.9087
Epoch 104/1000
66414/66414 [==============================] - 25s 370us/step - loss: 0.2472 - acc: 0.9115 - val_loss: 0.2771 - val_acc: 0.9081
Epoch 105/1000
66414/66414 [==============================] - 25s 369us/step - loss: 0.2468 - acc: 0.9119 - val_loss: 0.2774 - val_acc: 0.9078
Epoch 106/1000
66414/66414 [==============================] - 25s 369us/step - loss: 0.2469 - acc: 0.9119 - val_loss: 0.2777 - val_acc: 0.9085
Epoch 107/1000
66414/66414 [==============================] - 25s 369us/step - loss: 0.2446 - acc: 0.9123 - val_loss: 0.2784 - val_acc: 0.9070
Epoch 108/1000

Epoch 13/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2852 - acc: 0.9086 - val_loss: 0.2787 - val_acc: 0.9162.2853 - ac
Epoch 14/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2827 - acc: 0.9086 - val_loss: 0.2726 - val_acc: 0.9162
Epoch 15/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2825 - acc: 0.9085 - val_loss: 0.2704 - val_acc: 0.9162
Epoch 16/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2813 - acc: 0.9087 - val_loss: 0.2741 - val_acc: 0.9164
Epoch 17/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2810 - acc: 0.9087 - val_loss: 0.2782 - val_acc: 0.9162
Epoch 18/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2799 - acc: 0.9088 - val_loss: 0.2765 - val_acc: 0.9164
Epoch 19/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2794 - acc: 0.9083 - val_loss: 0.2731 - val_acc: 0.9

Epoch 69/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2563 - acc: 0.9099 - val_loss: 0.2696 - val_acc: 0.9162
Epoch 70/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2556 - acc: 0.9097 - val_loss: 0.2689 - val_acc: 0.9164
Epoch 71/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2578 - acc: 0.9093 - val_loss: 0.2664 - val_acc: 0.9158
Epoch 72/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2563 - acc: 0.9095 - val_loss: 0.2693 - val_acc: 0.9161
Epoch 73/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2541 - acc: 0.9101 - val_loss: 0.2747 - val_acc: 0.9138
Epoch 74/1000
66414/66414 [==============================] - 24s 366us/step - loss: 0.2539 - acc: 0.9101 - val_loss: 0.2742 - val_acc: 0.9150
Epoch 75/1000
66414/66414 [==============================] - 24s 367us/step - loss: 0.2524 - acc: 0.9098 - val_loss: 0.2677 - val_acc: 0.9161
Epoch 

In [22]:
auc_mean

[0.6942366965152929,
 0.6731511797644351,
 0.6740070511822408,
 0.6852779419701371,
 0.6717783844231022,
 0.6790888093807175,
 0.6747679110780535,
 0.6863164032333371,
 0.6880980594876558,
 0.6575320860312022]

In [26]:
model = load_model('C:/Users/Crow/Desktop/result/re_CNN3/model/3checkpoint-0.29-50e-val_acc_0.91.hdf5')

test_pred_proba = model.predict(x_test)
test_pred = model.predict_classes(x_test)
fpr, tpr, thresholds = roc_curve(y_test,test_pred_proba,pos_label=1)
print("ACC:  %f "  %accuracy_score(y_test,test_pred))
print("AUC: %f" % auc(fpr, tpr))

ACC:  0.917464 
AUC: 0.662739


In [3]:
np.arange(0,0.11,0.01)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ])